# Autonomy fleet visualization

The objectives of this notebook are to :
<ol>
    <li> Download autonomy assets data </li>
    <li> Show the current location of autonomy assets </li>
    <li> Show the last profiles of the assets </li>
    <li> Show the full depth/time transect of the assets </li>
</ol>

To run this notebook please refer to the readme of the github page (https://github.com/NOC-OBG-Autonomy/biocarbon_nrt_data_viz).

## Download autonomy assets data

In [ ]:
import pandas as pd
import numpy as np
import requests
import shutil
import os
import gzip
from pathlib import Path
import re
from tqdm import tqdm
from urllib.request import urlretrieve
import xarray as xr
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from glob import glob

We build the data repository structure. The data folder is not sync with git, so we need to create it if missing (i.e. first time your run this notebook on your computer).

In [ ]:
def create_missing_directories():
    # Define the path to the parent directory
    parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

    # Check if 'data' folder exists in the parent directory
    data_dir = os.path.join(parent_dir, 'data')
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
        print("'data' folder created in the parent directory.")
    else:
        print("'data' folder already exists in the parent directory.")

    # Check if 'floats' directory exists inside 'data' folder
    floats_dir = os.path.join(data_dir, 'Floats')
    if not os.path.exists(floats_dir):
        os.makedirs(floats_dir)
        print("'floats' directory created inside 'data' folder.")
    else:
        print("'floats' directory already exists inside 'data' folder.")

    # Check if 'gliders' directory exists inside 'data' folder
    gliders_dir = os.path.join(data_dir, 'Gliders')
    if not os.path.exists(gliders_dir):
        os.makedirs(gliders_dir)
        print("'gliders' directory created inside 'data' folder.")
    else:
        print("'gliders' directory already exists inside 'data' folder.")

if __name__ == "__main__":
    create_missing_directories()

For now, we will visualize only test floats. One in the Icelandic bassin and the other in SO during the Custard cruise, which is a good test case as it has also gliders deployements.

We don't automate in this script the retrieving of downloading url for the sake of simplicity. 
You can keep in mind that the list of float data and their repertory are findable here : https://data-argo.ifremer.fr/argo_synthetic-profile_index.tx

In [ ]:
wmo_list = [4903532, 5906213]
#Float 1 = test float in the Icelandic Bassin
float_1_url = 'https://data-argo.ifremer.fr/dac/aoml/4903532/4903532_Sprof.nc'
#Float 2 = test float on Custard with glider next to it
float_2_url = 'https://data-argo.ifremer.fr/dac/aoml/5906213/5906213_Sprof.nc'

#List the floats
floats_url = [float_1_url, float_2_url]

#Assign the local float directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
floats_dir =  os.path.join(parent_dir, 'Data/Floats')

#Create floats filename
floats_filenames = []
for i in floats_url:
    filename = floats_dir + '/' + i.rsplit('/', 1)[1]
    floats_filenames.append(filename)

In [ ]:
#Download floats data. We do it everytime so we are sure to work with a DAC up to date version. 
#We could check with the synthetic profile index if it is needed to download a update.
for url, filename in zip(floats_url, floats_filenames):
    urlretrieve(url, filename)


## Plot BGC-Argo locations

### Icelandic Bassin float

In [ ]:
print(wmo_list)

In [ ]:
position_df = pd.DataFrame({'PROF_NUM' : str(), 'LONGITUDE' : [], 'LATITUDE' : [], 'float' : int()})
for file, wmo in zip(floats_filenames, wmo_list):
    dat = xr.open_dataset(file)
    dat = dat.rename({'CYCLE_NUMBER':'PROF_NUM'}).swap_dims({'N_PROF':'PROF_NUM'})
    temp_df = dat[['LONGITUDE', 'LATITUDE']].to_dataframe().reset_index()
    temp_df['float'] = wmo
    position_df = pd.concat([position_df, temp_df], ignore_index=True)
    dat.close()
    

In [ ]:
# load the low resolution world map
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))

In [ ]:
# initialize an empty figure and add an axis
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot()

# plot a basic map of the world
world.plot(
    ax=ax,
    color="lightgray",
    edgecolor="black",
    alpha=0.5
)

plt.scatter(position_df['LONGITUDE'], position_df['LATITUDE'], c = position_df['PROF_NUM'])

# turn off axis ticks
ax.set_xticks([])
ax.set_yticks([])

#set the plot color bar
cbar = plt.colorbar(label='Profile Number')
cbar.set_label('Profile Number', rotation=270, labelpad=15)

# set the plot title
plt.title("Map of the BGC-Argo floats")
plt.show()

Zoom on the Icelandic bassin

In [ ]:
# initialize an empty figure and add an axis
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot()

# plot a basic map of the world
world.plot(
    ax=ax,
    color="lightgray",
    edgecolor="black",
    alpha=0.5
)

iceland_df = position_df[position_df['float'] == 4903532]
plt.scatter(iceland_df['LONGITUDE'], iceland_df['LATITUDE'], c = iceland_df['PROF_NUM'])
plt.plot(iceland_df['LONGITUDE'], iceland_df['LATITUDE'], c = 'black')

#set boundaries
min_lon = min(iceland_df['LONGITUDE'])
max_lon = max(iceland_df['LONGITUDE'])

min_lat = min(iceland_df['LATITUDE'])
max_lat = max(iceland_df['LATITUDE'])

# turn off axis ticks
ax.set_xticks([])
ax.set_yticks([])

plt.xlim([min_lon - 3, max_lon + 3])
plt.ylim([min_lat - 1, max_lat + 1])
plt.gca().set_aspect('equal', adjustable='datalim')

#set the plot color bar
cbar = plt.colorbar(label='Profile Number')
cbar.set_label('Profile Number', rotation=270, labelpad=15)

# set the plot title
plt.title("Map of the BGC-Argo floats")
plt.show()

In [ ]:
bath_directory = parent_dir + '/Data/' + "ne_10m_bathymetry_all/"
def load_bathymetry(zip_file_url):
    """Read zip file from Natural Earth containing bathymetry shapefiles"""
    # Download and extract shapefiles
    import io
    import zipfile

    r = requests.get(zip_file_url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(bath_directory)

    # Read shapefiles, sorted by depth
    shp_dict = {}
    files = glob(bath_directory + '*.shp')
    assert len(files) > 0
    files.sort()
    depths = []
    for f in files:
        depth = '-' + f.split('_')[-1].split('.')[0]  # depth from file name
        depths.append(depth)
        bbox = (min_lon - 3, max_lon + 3,min_lat - 1, max_lat + 1)  # (x0, y0, x1, y1)
        nei = shpreader.Reader(f, bbox=bbox)
        shp_dict[depth] = nei
    depths = np.array(depths)[::-1]  # sort from surface to bottom
    return depths, shp_dict

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
import matplotlib
if __name__ == "__main__":
    # Load data (14.8 MB file)
    depths_str, shp_dict = load_bathymetry(
        'https://naturalearth.s3.amazonaws.com/' +
        '10m_physical/ne_10m_bathymetry_all.zip')

    # Construct a discrete colormap with colors corresponding to each depth
    depths = depths_str.astype(int)
    N = len(depths)
    nudge = 0.01  # shift bin edge slightly to include data
    boundaries = [min(depths)] + sorted(depths+nudge)  # low to high
    norm = matplotlib.colors.BoundaryNorm(boundaries, N)
    blues_cm = matplotlib.colormaps['Blues_r'].resampled(N)
    colors_depths = blues_cm(norm(depths))

    # Set up plot
    subplot_kw = {'projection': ccrs.LambertCylindrical()}
    fig, ax = plt.subplots(subplot_kw=subplot_kw, figsize=(9, 7))
    ax.set_extent([min_lon - 3, max_lon + 3,min_lat - 5, max_lat + 5], crs=ccrs.PlateCarree())  # x0, x1, y0, y1

    # Iterate and plot feature for each depth level
    for i, depth_str in enumerate(depths_str):
        ax.add_geometries(shp_dict[depth_str].geometries(),
                          crs=ccrs.PlateCarree(),
                          color=colors_depths[i])

    # Add standard features
    #ax.add_feature(cfeature.LAND, color='grey')
    ax.coastlines(lw=1, resolution='110m')
    ax.gridlines(draw_labels=False)
    ax.set_position([0.03, 0.05, 0.8, 0.9])

    
    # Convert vector bathymetries to raster (saves a lot of disk space)
    # while leaving labels as vectors
    ax.set_rasterized(True)
    plt.scatter(iceland_df['LONGITUDE'], iceland_df['LATITUDE'], c = iceland_df['PROF_NUM'])
    plt.plot(iceland_df['LONGITUDE'], iceland_df['LATITUDE'], c = 'black')
